#Auto-ESN

In [ ]:
!pip install syft==0.2.9

In [ ]:
!pip install auto-esn

In [ ]:
import torch
import syft as sy
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn
import torch.optim as optim
from auto_esn.esn.esn import DeepESN
#from auto_esn.esn.reservoir.util import NRMSELoss

class CarHackingDataset(Dataset):
    """
    Loading the Car-Hacking Dataset from
    https://ocslab.hksecurity.net/Datasets/car-hacking-dataset 

    Args:
        csv_file: A path to the dataset file which has the extension CSV.
        root_dir: The directory of the parent folder of the dataset.
        transform (callable, optional): Optional tansform to be applied on a sample.
    """
    def __init__(self, csv_file: str, root_dir: str, transform=None):
        self.car_hacking_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self,idx):
        '''Grabs relevant features from the dataset.'''
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = ['Timestamp', 'DLC', 'CAN_ID', 'Data']
        X_train = self.car_hacking_frame.loc[:, features].values
        X_train = torch.as_tensor(X_train)

        # It looks like it's a bad idea to encode features.
        # https://stackoverflow.com/questions/61217713/labelencoder-for-categorical-features

        y = self.car_hacking_frame['Flag'].iloc[idx]
        #y = torch.as_tensor(y)

        return X_train[idx], y
            
    def __len__(self):
        return len(self.car_hacking_frame)

class ValidationDataset(Dataset):
    def __init__(self, csv_file: str, root_dir: str, transform=None):
        self.validation_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self, idx):
        '''Grabs relevant features from the dataset.'''
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = ['Timestamp', 'DLC', 'CAN_ID', 'Data']
        X_train = self.validation_frame.loc[:, features].values
        X_train = torch.as_tensor(X_train, dtype=torch.float32)
        y = self.validation_frame['Flag'].iloc[idx]
        y = torch.as_tensor(y, dtype=torch.float32)

        return X_train[idx], y

    def __len__(self):
        return len(self.validation_frame)

train_dataset = CarHackingDataset(csv_file='/content/car_hacking_data/split_fuzzy_dataset_2_training_imbalanced.csv', 
                                  root_dir='/content/car_hacking_data')

test_loader = ValidationDataset(csv_file='/content/car_hacking_data/split_fuzzy_dataset_2_testing_imbalanced.csv', 
                                 root_dir='/content/car_hacking_data')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hook = sy.TorchHook(torch)

car1 = sy.VirtualWorker(hook, id="car1")
car2 = sy.VirtualWorker(hook, id="car2")

args = {
    'batch_size' : train_dataset.__len__(),
    'epochs' : 5
}

federated_train_loader = sy.FederatedDataLoader(train_dataset.federate((car1, car2)),
                                                batch_size=args['batch_size'], shuffle=True, drop_last=True)


In [ ]:
import time
from sklearn.metrics import balanced_accuracy_score
import torch.nn as nn

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model = model.send(data.location)

        data, target = data.reshape(args['batch_size'], 4, -1).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()

        print(batch_idx)

        if batch_idx % 10 == 0:
            loss = loss.get()

            print(f'''Train Epoch: {epoch} [{(batch_idx * args['batch_size'])}/{(len(federated_train_loader) * args['batch_size'])}'''
                   + f'''({100. * batch_idx / len(federated_train_loader):.0f}%)]\tLoss: {loss.item():.6f}''')

def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0
    total = 0
    m = nn.Sigmoid()
    loss = nn.BCELoss()

    with torch.no_grad():
        for data, target in test_loader:
            #data, target = data.reshape(-1, 4, 1).to(device), target.to(device)
            data = data.reshape(-1, 1, 1).to(device)
            target = target.to(device)
            output = model(data)

            y = target.repeat(4, 1)
            test_loss += loss(m(data), y).item()

            #test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            # print(y.view_as(pred))
            y = y.reshape(4, 1)
            #correct += pred.eq(y.view_as(pred)).sum().item()
            total += y.size(0)
            correct += (pred == y).sum().item()

    test_loss /= len(test_loader)
    y = torch.as_tensor(y, dtype=torch.int8)

    print(f"The length of y is {len(y)}, shape of y is {y.shape}.")
    print(f"The pred is {len(pred)}, shape of pred is {pred.shape}")

    print(f"Test Loss: {test_loss}, Correct: {correct}, Accuracy: {100 * correct // total}%")

    '''print("\n Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
          test_loss, correct, test_loader.__len__,
          100. * correct/test_loader.__len__))'''

model = DeepESN(hidden_size=5000).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

t0 = time.time()
for epoch in range(1, args['batch_size'] + 1):
    train(model, device, federated_train_loader, optimizer, epoch)
t1 = time.time()

test(model, device, test_loader)
print(f'Training took {t1 - t0}s')

The length of y is 4, shape of y is torch.Size([4, 1]).
The pred is 4, shape of pred is torch.Size([4, 1])
Test Loss: 0.7534854360620181, Correct: 7836, Accuracy: 65%
Training took 12.636591911315918s


In [ ]:
import torch

test_loader = ValidationDataset(csv_file='/content/car_hacking_data/split_fuzzy_dataset_1_testing.csv', 
                                 root_dir='/content/car_hacking_data')

for data, target in test_loader:
  #print(f"OG Data: {data.reshape(-1, 1, 1)}, Target: {target}")
  print(data.reshape(-1, 1, 1))
  #print(target.size(0))
  #break


tensor([[[-0.5534]],

        [[ 0.1442]],

        [[ 0.1075]],

        [[-0.7168]]])
tensor([[[-0.2782]],

        [[ 0.1442]],

        [[-1.0058]],

        [[-0.5574]]])
tensor([[[-0.8290]],

        [[ 0.1442]],

        [[-1.3635]],

        [[ 0.2713]]])
tensor([[[-0.2666]],

        [[ 0.1442]],

        [[-1.0413]],

        [[ 1.9676]]])
tensor([[[-0.2832]],

        [[ 0.1442]],

        [[-0.8021]],

        [[ 2.0012]]])
tensor([[[-0.4874]],

        [[ 0.1442]],

        [[-1.0437]],

        [[-0.5554]]])
tensor([[[ 0.0247]],

        [[ 0.1442]],

        [[-0.1720]],

        [[-0.6860]]])
tensor([[[ 0.0940]],

        [[ 0.1442]],

        [[-1.0248]],

        [[ 1.7321]]])
tensor([[[-0.2178]],

        [[ 0.1442]],

        [[-1.0413]],

        [[-0.2304]]])
tensor([[[-0.4169]],

        [[ 0.1442]],

        [[-1.0579]],

        [[ 1.8224]]])
tensor([[[ 0.1729]],

        [[ 0.1442]],

        [[-1.0271]],

        [[ 2.0008]]])
tensor([[[-0.1149]],

        [[

KeyboardInterrupt: ignored

In [ ]:
print(test_loader.__len__())

7850


In [ ]:
for batch_idx, (data, target) in enumerate(federated_train_loader):
  #data = data.view(args['batch_size'], 1, 1)
  print(f"Data shape: {data.shape}, Target shape: {target.shape}")

In [ ]:
"""for (data, target) in test_loader:
  print(f"Data: {data}, Target: {target}")
"""

for data in test_loader:
  print(data)

"""v1 = ValidationDataset(csv_file='/content/car_hacking_data/clean_scaled_normal_run.csv', root_dir='/content/car_hacking_data')
print(v1.__getitem__(5))"""

tensor([-1.7320,  0.2429,  0.3913, -0.6454], dtype=torch.float64)
tensor([-1.7320e+00,  2.4294e-01,  3.8467e-08, -4.8861e-01],
       dtype=torch.float64)
tensor([-1.7320,  0.2429,  0.9999, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429,  1.3504, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429, -0.5624, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429, -0.9917, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429, -1.9074, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429, -0.8287,  1.9859], dtype=torch.float64)
tensor([-1.7320,  0.2429, -1.0868, -0.6629], dtype=torch.float64)
tensor([-1.7320,  0.2429, -1.0841, -0.6946], dtype=torch.float64)
tensor([-1.7320,  0.2429, -1.0434, -0.6999], dtype=torch.float64)
tensor([-1.7320,  0.2429, -0.2608, -0.4132], dtype=torch.float64)
tensor([-1.7320,  0.2429, -0.0869,  0.3144], dtype=torch.float64)
tensor([-1.7320,  0.2429,  0.2337, -0.6456], dtype=torch.float64)
tensor([-1.7320,  0.2429,  0.2853,  0.7340], dtype=to

KeyboardInterrupt: ignored

# Model Shape Information using TorchInfo

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchinfo import summary
from auto_esn.esn.esn import DeepESN

model2 = DeepESN()
summary(model2, input_size=(100, 1, 1))

Layer (type:depth-idx)                   Output Shape              Param #
DeepESN                                  [100, 1]                  --
├─DeepESNCell: 1-1                       [100, 1000]               --
│    └─Activation: 2-1                   [1, 500]                  --
│    └─Activation: 2-2                   [1, 500]                  --
│    └─Activation: 2-3                   [1, 500]                  --
│    └─Activation: 2-4                   [1, 500]                  --
│    └─Activation: 2-5                   [1, 500]                  --
│    └─Activation: 2-6                   [1, 500]                  --
│    └─Activation: 2-7                   [1, 500]                  --
│    └─Activation: 2-8                   [1, 500]                  --
│    └─Activation: 2-9                   [1, 500]                  --
│    └─Activation: 2-10                  [1, 500]                  --
│    └─Activation: 2-11                  [1, 500]                  --
│    └─Activati

In [ ]:
a = torch.arange(4.)
a

tensor([0., 1., 2., 3.])

#Using PCA Results to Reduce Training Time


In [ ]:
import torch
import syft as sy
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import torch.nn
import torch.optim as optim
from auto_esn.esn.esn import DeepESN
#from auto_esn.esn.reservoir.util import NRMSELoss

class CarHackingDataset(Dataset):
    """
    Loading the Car-Hacking Dataset from
    https://ocslab.hksecurity.net/Datasets/car-hacking-dataset 

    Args:
        csv_file: A path to the dataset file which has the extension CSV.
        root_dir: The directory of the parent folder of the dataset.
        transform (callable, optional): Optional tansform to be applied on a sample.
    """
    def __init__(self, csv_file: str, root_dir: str, transform=None):
        self.car_hacking_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self,idx):
        '''Grabs relevant features from the dataset.'''
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = ['principal component 1', 'principal component 2']
        X_train = self.car_hacking_frame.loc[:, features].values
        X_train = torch.as_tensor(X_train)

        # It looks like it's a bad idea to encode features.
        # https://stackoverflow.com/questions/61217713/labelencoder-for-categorical-features

        return X_train[idx], self.car_hacking_frame['Flag'].iloc[idx]
            
    def __len__(self):
        return len(self.car_hacking_frame)

train_dataset = CarHackingDataset(csv_file='/content/car_hacking_data/balanced_pca_fuzzy_dataset.csv', 
                                  root_dir='/content/car_hacking_data')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hook = sy.TorchHook(torch)

car1 = sy.VirtualWorker(hook, id="car1")
car2 = sy.VirtualWorker(hook, id="car2")

args = {
    'batch_size' : 13084,
    'epochs' : 1
}

federated_train_loader = sy.FederatedDataLoader(train_dataset.federate((car1, car2)),
                                                batch_size=args['batch_size'], shuffle=True, drop_last=True)


In [ ]:
from auto_esn.esn.esn import DeepESN
import time

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model = model.send(data.location)

        data, target = data.reshape(args['batch_size'], 2, -1).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()

        if batch_idx % 10 == 0:
            loss = loss.get()

            print(f'''Train Epoch: {epoch} [{(batch_idx * args['batch_size'])}/{(len(federated_train_loader) * args['batch_size'])}'''
                   + f'''({100. * batch_idx / len(federated_train_loader):.0f}%)]\tLoss: {loss.item():.6f}''')

model = DeepESN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

t0 = time.time()
for epoch in range(1, args['batch_size'] + 1):
    train(model, device, federated_train_loader, optimizer, epoch)
t1 = time.time()
print(f'Training took {t1 - t0}s')

Training took 43.12057018280029s


#Changing the Loss Function and using NRMSE

In [ ]:
from auto_esn.esn.esn import DeepESN
from auto_esn.esn.reservoir.util import NRMSELoss
import time

nrmse = NRMSELoss()

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model = model.send(data.location)

        data, target = data.reshape(args['batch_size'], 2, -1).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = nrmse(output, target)
        loss.backward()
        optimizer.step()
        model.get()

        if batch_idx % 10 == 0:
            loss = loss.get()

            print(f'''Train Epoch: {epoch} [{(batch_idx * args['batch_size'])}/{(len(federated_train_loader) * args['batch_size'])}'''
                   + f'''({100. * batch_idx / len(federated_train_loader):.0f}%)]\tLoss: {loss.item():.6f}''')

model = DeepESN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

t0 = time.time()
for epoch in range(1, args['batch_size'] + 1):
    train(model, device, federated_train_loader, optimizer, epoch)
t1 = time.time()
print(f'Training took {t1 - t0}s')

Training took 41.67633080482483s


#Checking the Data inside Virtual Workers

In [ ]:
car1.msg_history

[]

#Testing Concatanation for DeepESN

In [ ]:
import torch

x = torch.randn(2, 3)
x

tensor([[-0.2088,  1.6677, -0.5772],
        [ 0.1312,  1.1963, -1.2029]])

In [ ]:
torch.cat((x, x, x), 0)


tensor([[-0.2088,  1.6677, -0.5772],
        [ 0.1312,  1.1963, -1.2029],
        [-0.2088,  1.6677, -0.5772],
        [ 0.1312,  1.1963, -1.2029],
        [-0.2088,  1.6677, -0.5772],
        [ 0.1312,  1.1963, -1.2029]])

In [ ]:
torch.cat((x, x, x), 1)


tensor([[-0.2088,  1.6677, -0.5772, -0.2088,  1.6677, -0.5772, -0.2088,  1.6677,
         -0.5772],
        [ 0.1312,  1.1963, -1.2029,  0.1312,  1.1963, -1.2029,  0.1312,  1.1963,
         -1.2029]])

#Tensor Manipulation

In [ ]:
import torch

x = torch.tensor([0.5, 0.2, 0.3, 0.4])
x = x.reshape(4,1)

y = torch.tensor([1.])
y = y.repeat(4, 1)
print(y)


m = nn.Sigmoid()
loss = nn.BCELoss()
output = loss(m(x), y)
print(output)

tensor([[1.],
        [1.],
        [1.],
        [1.]])
tensor(0.5349)


#Loss Function Testing

In [ ]:
import torch.nn as nn

m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
print(input)
target = torch.empty(3).random_(3)
print(target)
output = loss(m(input), target)
print(output) #tensor(0.4198, grad_fn=<BinaryCrossEntropyBackward>)


tensor([ 0.7086, -1.0160, -0.2988], requires_grad=True)
tensor([0., 1., 1.])
tensor(1.0959, grad_fn=<BinaryCrossEntropyBackward>)


In [ ]:
target_1 = torch.tensor([0.0])
correct = 0


correct += pred.eq(target.view_as(pred)).sum().item()

#Using EchoTorch for ESN

In [ ]:
!pip install EchoTorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for EchoTorch: filename=EchoTorch-0.1.1-py3-none-any.whl size=55562 sha256=452a0fd0929278bca13e4ab0c13c8521612f7b482ec25473e0f01ef40974192f
  Stored in directory: /root/.cache/pip/wheels/69/fc/7c/c7d0886395ed9b689a4c5442ea99d3c968768667163297e276
Successfully built EchoTorch


In [ ]:


esn = etnn.LiESN()

# Using PyESN

In [ ]:
from pyESN import ESN
from torchinfo import summary

esn = ESN(n_inputs = 4,
          n_outputs = 1,
          n_reservoir = 10,
          spectral_radius = 1.5,
          random_state=42)

summary(esn, input_size=(10, 1))

AttributeError: ignored

In [ ]:
import torch
import syft as sy
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn
import torch.optim as optim

class CarHackingDataset(Dataset):
    """
    Loading the Car-Hacking Dataset from
    https://ocslab.hksecurity.net/Datasets/car-hacking-dataset 

    Args:
        csv_file: A path to the dataset file which has the extension CSV.
        root_dir: The directory of the parent folder of the dataset.
        transform (callable, optional): Optional tansform to be applied on a sample.
    """
    def __init__(self, csv_file: str, root_dir: str, transform=None):
        self.car_hacking_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self,idx):
        '''Grabs relevant features from the dataset.'''
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = ['Timestamp', 'DLC', 'CAN_ID', 'Data']
        X_train = self.car_hacking_frame.loc[:, features].values
        X_train = torch.as_tensor(X_train)

        # It looks like it's a bad idea to encode features.
        # https://stackoverflow.com/questions/61217713/labelencoder-for-categorical-features

        y = self.car_hacking_frame['Flag'].iloc[idx]
        y = torch.as_tensor(y)

        return X_train[idx], y
            
    def __len__(self):
        return len(self.car_hacking_frame)

class ValidationDataset(Dataset):
    def __init__(self, csv_file: str, root_dir: str, transform=None):
        self.validation_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self, idx):
        '''Grabs relevant features from the dataset.'''
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = ['Timestamp', 'DLC', 'CAN_ID', 'Data']
        X_train = self.validation_frame.loc[:, features].values
        X_train = torch.as_tensor(X_train, dtype=torch.float32)
        y = self.validation_frame['Flag'].iloc[idx]
        y = torch.as_tensor(y, dtype=torch.float32)

        return X_train[idx], y

    def __len__(self):
        return len(self.validation_frame)

train_dataset = CarHackingDataset(csv_file='/content/car_hacking_data/split_fuzzy_dataset_1_training.csv', 
                                  root_dir='/content/car_hacking_data')

test_loader = ValidationDataset(csv_file='/content/car_hacking_data/split_fuzzy_dataset_1_testing.csv', 
                                 root_dir='/content/car_hacking_data')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hook = sy.TorchHook(torch)

car1 = sy.VirtualWorker(hook, id="car1")
car2 = sy.VirtualWorker(hook, id="car2")

args = {
    'batch_size' : train_dataset.__len__(),
    'epochs' : 15
}

federated_train_loader = sy.FederatedDataLoader(train_dataset.federate((car1, car2)),
                                                batch_size=args['batch_size'], shuffle=True, drop_last=True)


In [ ]:
import time
from sklearn.metrics import balanced_accuracy_score
import torch.nn as nn

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model = model.send(data.location)

        data, target = data.reshape(args['batch_size'], 4, -1).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()

        print(batch_idx)

        if batch_idx % 10 == 0:
            loss = loss.get()

            print(f'''Train Epoch: {epoch} [{(batch_idx * args['batch_size'])}/{(len(federated_train_loader) * args['batch_size'])}'''
                   + f'''({100. * batch_idx / len(federated_train_loader):.0f}%)]\tLoss: {loss.item():.6f}''')

def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0
    m = nn.Sigmoid()
    loss = nn.BCELoss()

    with torch.no_grad():
        for data, target in test_loader:
            #data, target = data.reshape(-1, 4, 1).to(device), target.to(device)
            data = data.reshape(-1, 1, 1).to(device)
            target = target.to(device)
            output = model(data)

            y = target.repeat(4, 1)
            test_loss = loss(m(data), y)

            #test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            # print(y.view_as(pred))
            y = y.reshape(4, 1, -1)
            #correct += pred.eq(y.view_as(pred)).sum().item()

    #test_loss /= test_loader.__len__
    print(f"Test Loss: {test_loss}, Correct: {correct}, Accuracy: {balanced_accuracy_score(y, pred)}")

    '''print("\n Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
          test_loss, correct, test_loader.__len__,
          100. * correct/test_loader.__len__))'''

model = ESN(n_inputs = 4,
            n_outputs = 1,
            n_reservoir = 10,
            spectral_radius = 1.5,
            random_state=42).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

t0 = time.time()
for epoch in range(1, args['batch_size'] + 1):
    train(model, device, federated_train_loader, optimizer, epoch)
    test(model, device, test_loader)
t1 = time.time()
print(f'Training took {t1 - t0}s')